In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np 
#for importing files
import glob, os
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import math

#Library for hull calculation
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt
from matplotlib.path import Path
from scipy.spatial import distance


## Helper Methods

In [2]:
def SecsToGameTime(time):
    mins=(int(time)/60)
    secs=int(time)%60
    print (str(mins)+":"+str(secs))
    return str(mins)+":"+str(secs)

In [3]:
def GameTimetoSeconds(time):
    mins, secs = time.split(":")
    timeSecs = int(mins)*60+int(secs)
    #print(timeSecs)
    return timeSecs

#### Hull Calculation Helper Method

In [4]:
def getHull(moments, teamID):
    points = []
    
    if(int(moments[5][1][0]) == teamID):
        for positions in moments[5][1:6]:
            #print positions
            points.append([positions[2],positions[3]])
        print("Defensive positions: "+ str(points))
    else:
        for positions in moments[5][6:11]:
            #print positions
            points.append([positions[2],positions[3]])
        print("Defensive positions: "+str(points))
    
    np_points = np.asarray(points)
    np_points
    hull = ConvexHull(np_points)
    
    return hull,np_points

In [5]:
def distFromCentroid(hull, x,y):
        cx = np.mean(hull.points[hull.vertices,0])
        cy = np.mean(hull.points[hull.vertices,1])
        return ((x-cx)^2+(y-cy)^2)^.5

In [6]:
def isInHull(hull,np_points,x,y):
    #point in hull?
    hull_path = Path( np_points[hull.vertices] )
    isInPath = hull_path.contains_point((x,y))
    return (isInPath == True)

### Helper method for returning PBP descriptions given game and eventNum

In [7]:
def getPBPInfo(gameID,PBPEventNum):
    curPath = os.getcwd()
    if(not curPath.endswith('\pbp-csv')):
        curPath = os.getcwd()
        print curPath
        strIndex=curPath.index('\\JupyterNotebooks')
        curPath = curPath[:(strIndex+18)]+'\pbp-csv'
        os.chdir(curPath)
        print curPath
    
    events_df = pd.read_csv("00"+str(gameID)+".csv")
    eventRow = events_df[events_df['EVENTNUM'].isin([int(PBPEventNum)])]
     
    print eventRow['PCTIMESTRING']
    print eventRow['HOMEDESCRIPTION']
    print eventRow['VISITORDESCRIPTION']
    return eventRow['PCTIMESTRING'] , eventRow['HOMEDESCRIPTION'] , eventRow['VISITORDESCRIPTION']
    

### Given the game time (12:00) and eventNumber from the Play-by-play data for a recorded event, this method returns the correct eventNumber from the SportsVU data since they actually dont matchup.  This method converts the gametime into seconds (720s = 12:00) which is formatted in SportsVU.  It traverses up and down events until the gametime is within a selected eventNumber.

In [26]:
def findCorrectMomentData(rawSportVUData, quarterPBP, gametimePBP, approxEventNumPBP):
    print("Finding correct SportVU eventNum for gameid:"+rawSportVUData['gameid'])

    found = False;
    moments = None
    
    checkedValues = []
    
    prevEventNumPBP = None
    #convert gametimePBP (12:00) to seconds (720)
    gameInSeconds = GameTimetoSeconds(gametimePBP)
    while (found == False):
        print ("curr eventNum:"+str(approxEventNumPBP)+" , prev eventNum:"+ str(prevEventNumPBP))
        
        if(approxEventNumPBP in checkedValues):
            print("infinite loop")
            return None, None
        else:
            checkedValues.append(approxEventNumPBP)        
        
        print("Currently checking eventNumPBP: "+str(approxEventNumPBP))
        moments = MomentsfromEventNum(rawSportVUData, approxEventNumPBP)
        if (moments is None or len(moments) == 0):
            print("Could not find moments for this eventNum in SportsVU: "+ str(approxEventNumPBP))
            return None, None
        
        quarterSportVU = int(moments[0][0])
        print("Comparing Target PBP quarter:"+str(quarterPBP)+" , with  SportVU quarter :"+str(quarterSportVU))
        print("Comparing Target PBP gameclock:"+str(gameInSeconds)+" with SportsVU moment game clock:"+str(moments[0][2]))
        
        #check whether this EventNum moment is the correct time frame of the play from Play-by-play
        if(quarterPBP < quarterSportVU):
            approxEventNumPBP -= 1
        elif(quarterPBP > quarterSportVU):    
            approxEventNumPBP  += 1
            
        #is the PBP game time between the gametime of the moments extracted?
        elif(int(moments[0][2])+1 > gameInSeconds and int(moments[len(moments)-1][2])-1 < gameInSeconds):
            found=True
        elif(moments[0][2] < gameInSeconds):
            approxEventNumPBP -= 1
        elif(moments[len(moments)-1][2] > gameInSeconds):
        # look ahead 1 event 
            approxEventNumPBP += 1
        else:
            print("huge error, should not reach here")
            return None, None
        
    #the returned event num corresponds to SportsVU, which holds the time of the event we are looking for.... 
    print("Corrected eventNum to be searching for in SportsVU: "+ str(approxEventNumPBP))
    return  moments, approxEventNumPBP

## Method returns the moments for the eventNum from PBP

In [21]:
def MomentsfromEventNum(eventsData, eventNumPBP):
    found = False
    searchDirection = None
    sportVUIndex = eventNumPBP
    eventsLen = len(eventsData['events'])-1
    if(eventsLen < sportVUIndex):
        sportVUIndex = eventsLen
    
    while(not found):        
        if( int(eventsData['events'][sportVUIndex]['eventId']) == eventNumPBP):
            found = True
        elif ( int(eventsData['events'][sportVUIndex]['eventId']) > eventNumPBP ):
            if(searchDirection is None and sportVUIndex > 0):
                searchDirection = "decrementIndex"
                sportVUIndex -= 1  
            elif(searchDirection is "decrementIndex" and sportVUIndex > 0):
                sportVUIndex -= 1  
            elif(searchDirection is "incrementIndex"):
                print("Failed finding eventnum in SportsVU, search directions switched.")
                return None
            else: #we have reached 0th index and cant traverse up the list anymore
                print("Failed finding eventnum in SportsVU")
                return None
        elif ( int(eventsData['events'][sportVUIndex]['eventId']) < eventNumPBP ):
            if(searchDirection is None and sportVUIndex < eventsLen):
                searchDirection = "incrementIndex"
                sportVUIndex += 1  
            elif(searchDirection is "incrementIndex" and sportVUIndex < eventsLen):
                sportVUIndex += 1  
            elif(searchDirection is "decrementIndex"):
                "Failed finding eventnum in SportsVU, search directions switched."
                return None
            else: #we have reached 0th index and cant traverse up the list anymore
                print("Failed finding eventnum in SportsVU")
                return None
    print("SportVU Index for the eventNumPBP: "+str(sportVUIndex))
    
    return (eventsData['events'][sportVUIndex]["moments"]) , sportVUIndex

## Identify Moment of CATCH

1.  //sort events moment data from end time to beginning (reverse order)
timeballReceived, locationballReceived = getReceive (recieverID, List[Moments]) 

Each moment is taken every .25s, so the FIRST 8 moments (8 *.25 = 2 seconds) when the receiverID and ball are within (dist <= 1 ft), is when the ball was the moment before the ball was released.

reset numCount = 0;

Starting from this time, continue iterating through moments. If dist from receiver and ball >=1 ft, then this is time the ball was caught.

***separate calculation to check when the ball dist less than 1 ft for another teammate to determine if it was a pass in a certain amount of time (counted numCount)****

2.  Find "latest" instance when   distFrom ( Moment.Positions[0].x,Moment.Position[0].y , Moment.Position[ ??=playerID ].x , Moment.Position[ ??=playerID].y) <= 1ft )

   return the timemilis ,  position[0].x , position[0].y


In [10]:
 def getCatchEventDetails(receiverID, passerID, gametimePBP, moments):
    momentsReversed = moments[::-1]
    processComplete = False;
    indexBallCaught = None
    indexBallReleased = None
    finalIndex=0
    index=0
    found = False
    ball2receiver_distThreshold = 1 #if ball is < 1 feet from receiver, then he has it
    catchball2receiver_distThreshold = 2 # dist threshold for determining when the ball is caught has to be x ft away
    gameInSeconds = GameTimetoSeconds(gametimePBP)
    print("num of moments: "+str(len(momentsReversed)))
    while (found == False):
        #reverse the moment data, and 
        if(int(momentsReversed[index][2]) >=  gameInSeconds):
            found=True
        elif(int(momentsReversed[index][2]< gameInSeconds)):
            index += 1
        else:
            return "ERROR"
    
    print("starting at time: "+ str(momentsReversed[index][2])+ "with total momentList length of "+str(len(momentsReversed)))
    
    #Found the index of the approx start of the event
    numCounts=0
    for i in range(index,len(momentsReversed)):
       #Debug #print("Time of moment being checked"+ str(momentsReversed[i][2]))
       #Debug #print("current Index in momentList: "+ str(i))
        if numCounts >= 3:
            timeBallReleased = momentsReversed[i][2]
            #SecsToGameTime(timeBallReleased)  
            indexBallReleased = i
            break
        else:
            ball = momentsReversed[i][5][0]
            player_x = None
            player_y = None
            for positions in momentsReversed[i][5]:    
                if positions[1] == receiverID:
                    player_x = positions[2]
                    player_y = positions[3]
            
            if(player_x is None and player_y is None):
                print("couldnt find receiver in moment's position")
                continue
            else:
                dist = distance.euclidean([player_x,player_y],[ball[2],ball[3]])
                if dist <= ball2receiver_distThreshold:
                    print("receiver holding ball @ shotclock: " + str( momentsReversed[i][3]) + " @ gametime "+ str( momentsReversed[i][2] ))
                    numCounts += 1
    
    if(indexBallReleased is None):            
        print("Could not find when release occured, reached end of moment list and could not find when ball left receiver's hands")
        return None, None, None, None, None, None, None, None, None, None
    
    #count when ball is caught
    numCounts=0
    for j in range(indexBallReleased, len(momentsReversed)):
        if numCounts >= 3:
            timeBallCaught = momentsReversed[j][2]
            #SecsToGameTime(timeBallCaught)
            indexBallCaught = j
            
            #record position of receiver
            receiverMoment = momentsReversed[j][5]
            
            reciever_pos_x = None
            receiver_pos_y = None
            for positions in receiverMoment:
                if positions[1] == receiverID:
                    reciever_pos_x = positions[2]
                    receiver_pos_y = positions[3]
            
            if(reciever_pos_x is None and receiver_pos_y is None):
                continue
            
            receiverPosAtCaught = [reciever_pos_x, receiver_pos_y]
            print(receiverPosAtCaught)
            
            break
        else:
            ball = momentsReversed[j][5][0]
            player_x = None
            player_y = None
            for positions in momentsReversed[j][5]:
                if positions[1] == receiverID:
                    player_x = positions[2]
                    player_y = positions[3]
            
            #check if receiver was actually found in moment's positions
            if(player_x is None and player_y is None):
                continue
            else:
                dist = distance.euclidean([player_x,player_y],[ball[2],ball[3]])
                if dist >= 2:
                    print("receiver caught ball @ shotclock: " + str(momentsReversed[j][3])+" @ gametime " + str(momentsReversed[j][2]) + " received reversed index"+str(j))
                    #SecsToGameTime(momentsReversed[j][3])
                    numCounts += 1
            
            #if(momentsReversed[j][3] is None):       
            #    print("Couldnt find catch event")
            #   return None, None, None, None, None, None, None, None, None, None   
    
    #check if ball caught event in moments list
    if(indexBallCaught is None):      
        print("Couldnt find catch event")
        return None, None, None, None, None, None, None, None, None, None   
                
                
    #count when ball is passed
    numCounts=0
    for k in range(indexBallCaught, len(momentsReversed)):
        #Debug#print("Time of moment being checked"+ str(momentsReversed[k][2]))
        #Debug#print("current Index in momentList: "+ str(k))
        if numCounts >= 2:
            #Debug# print("passer passed ball @ shotclock: " +str(momentsReversed[k][3])+" @ gametime " + str(momentsReversed[k][2]))
            timeBallPassed = momentsReversed[k][2]
            indexBallPassed = k
            processComplete = True
               
            #record position of receiver
            receiverMoment = momentsReversed[k][5]
            receiverPos = [k for k in receiverMoment if receiverID in k]
            
            reciever_pos_x = receiverPos[0][2]
            receiver_pos_y = receiverPos[0][3]
            receiverPosAtPass = [reciever_pos_x ,receiver_pos_y]
        
            #record position of passer
            passerPos = [k for k in receiverMoment if passerID in k]
            
            passer_pos_x = passerPos[0][2]
            passer_pos_y = passerPos[0][3]
            passerPosAtPass = [passer_pos_x ,passer_pos_y]
            
            SportsVUReversedIndex = len(momentsReversed) - indexBallPassed
            
            break
        else:
            ball = momentsReversed[k][5][0]
            player_x = None
            player_y = None
            for positions in momentsReversed[k][5]:
                if positions[1] == passerID:
                    player_x = positions[2]
                    player_y = positions[3]
            if(player_x is None and player_y is None):
                continue
            else:
                dist = distance.euclidean([player_x,player_y],[ball[2],ball[3]])
                if dist <= 2:
                    #Debug#print("passer just passed ball @ shotclock " +str(momentsReversed[k][3])+" @ gametime " + str(momentsReversed[k][2]) + " passed reversed index" +str(k))
                    numCounts += 1
              
    if(not processComplete):
        print("Couldnt find pass event")
        return None, None, None, None, None, None, None, None, None , None
    
        
    return timeBallReleased, indexBallReleased, timeBallCaught, indexBallCaught, timeBallPassed, indexBallPassed, receiverPosAtPass, receiverPosAtCaught, passerPosAtPass, SportsVUReversedIndex 
   

## Get TO Event Info

In [23]:
 def getTOEventDetails(receiverID, passerID, defensiveTeamId, gametimePBP, moments):
    momentsReversed = moments[::-1]
    #since a recorded TO doesnt occur till the ball hits out of bounds, then we should delay getting the positions a few seconds before the recording to capture moment of pass
    secondsDelay  = 2
    processComplete = False;
    indexBallCaught = None
    indexBallReleased = None
    finalIndex=0
    index=0
    found = False
    gameInSeconds = GameTimetoSeconds(gametimePBP) + secondsDelay
    print("num of moments: "+str(len(momentsReversed)))
    while (found == False):
        if(int(momentsReversed[index][2]) >=  gameInSeconds):
            found=True
        elif(int(momentsReversed[index][2]< gameInSeconds)):
            index += 1
        else:
            return "ERROR"
    
    print("starting at time: "+ str(momentsReversed[index][2]))
    
    timeBallReleased = momentsReversed[index][2]
    indexBallReleased = index
    timeBallCaught = momentsReversed[index][2]
    timeBallPassed = momentsReversed[index][2]
    indexBallPassed = index
    indexBallCaught = index
 
    
    #record position of passer
    receiverMoment = momentsReversed[index][5]
    passerPos = [k for k in receiverMoment if passerID in k]
    if(len(passerPos) is 0):
        print("couldnt find passer")
        return None, None, None, None, None, None, None, None, None, None 

    passer_pos_x = passerPos[0][2]
    passer_pos_y = passerPos[0][3]
    passerPosAtPass = [passer_pos_x ,passer_pos_y]

    SportsVUReversedIndex = len(momentsReversed) - indexBallPassed
    
    
    #check if there is a receiver???
    if (receiverID is not 0):
        receiverPos = [k for k in receiverMoment if receiverID in k]
        if(len(receiverPos) is 0):
            print("couldnt find receiver")
            return None, None, None, None, None, None, None, None, None, None 
        reciever_pos_x = receiverPos[0][2]
        receiver_pos_y = receiverPos[0][3]
        receiverPosAtCaught = [reciever_pos_x, receiver_pos_y]
        receiverPosAtPass = [reciever_pos_x, receiver_pos_y]
    else:
        receiverPosAtPass = [0,0]
        receiverPosAtCaught = [0,0]
        
     
    return timeBallReleased, indexBallReleased, timeBallCaught, indexBallCaught, timeBallPassed, indexBallPassed, receiverPosAtPass, receiverPosAtCaught, passerPosAtPass, SportsVUReversedIndex 
   

### Is receiving player open calculation

In [12]:
import math
# is_open
#   determines whether a player is "open" based on distance of nearest defender
#   note that the moment that the player shoots isn't determined in this function
# Params:
#   shooter_loc: location of the shooter at a given moment as a tuple (x, y)
#   def_locs: locations of each defender as list of tuples (x, y)
def is_open(shooter_loc, def_locs):

    dists = []
    for loc in def_locs:
        dist = math.hypot(shooter_loc[0] - loc[0], shooter_loc[1] - loc[1])
        if dist < 5:
            return dist, False
        dists.append(dist)
    dists = sorted(dists)
    return dists[0], True

shooter_loc = (6, 30)
#def_locs = [(56.27694, 25.51460), (73.64107, 25.48774), (48.6492, 35.26915), (48.46352, 14.55436), (47.51449, 24.36448)]
def_locs = [[37.42557, 15.22122], [5.85435, 29.07262], [31.67908, 38.87012], [45.11946, 31.22009], [12.7611, 43.81261]]
print(is_open(shooter_loc, def_locs))

(0.9387478825009401, False)


### Extracts Assists, Bad Passes, Alley Oops from csvs in current directory and saves them into separate csv files

In [24]:
AlleyOop_df = pd.DataFrame()
curPath = os.getcwd()
print curPath
if(not curPath.endswith('\pbp-csv')):
    curPath = os.getcwd()
    print curPath
    strIndex=curPath.index('\\JupyterNotebooks')
    curPath = curPath[:(strIndex+18)]+'\pbp-csv'
    os.chdir(curPath)
    print curPath
for pbpFile in glob.glob("*.csv"):
    print(pbpFile)
    #read file as df
    events_df = pd.read_csv(pbpFile)
    AlleyOopList= events_df[events_df['EVENTMSGTYPE'].isin([2]) & ( events_df['VISITORDESCRIPTION'].str.contains("Alley", na = False ) | events_df['HOMEDESCRIPTION'].str.contains("Alley", na = False )  ) ]
    AlleyOop_df = AlleyOop_df.append(AlleyOopList)
    
AlleyOop_df.to_csv("MissAlleyOop_Parsed.csv", sep='\t')

D:\DataMining-NBA-Repo\JupyterNotebooks\SportVU_JSON_Data
D:\DataMining-NBA-Repo\JupyterNotebooks\SportVU_JSON_Data
D:\DataMining-NBA-Repo\JupyterNotebooks\\pbp-csv
0021500001.csv
0021500002.csv
0021500003.csv
0021500004.csv
0021500005.csv
0021500007.csv
0021500009.csv
0021500010.csv
0021500011.csv
0021500012.csv
0021500013.csv
0021500015.csv
0021500016.csv
0021500017.csv
0021500018.csv
0021500019.csv
0021500020.csv
0021500021.csv
0021500022.csv
0021500023.csv
0021500024.csv
0021500025.csv
0021500026.csv
0021500027.csv
0021500028.csv
0021500029.csv
0021500031.csv
0021500032.csv
0021500033.csv
0021500034.csv
0021500035.csv
0021500036.csv
0021500037.csv
0021500038.csv
0021500039.csv
0021500040.csv
0021500041.csv
0021500042.csv
0021500043.csv
0021500044.csv
0021500045.csv
0021500046.csv
0021500047.csv
0021500048.csv
0021500049.csv
0021500050.csv
0021500051.csv
0021500052.csv
0021500053.csv
0021500054.csv
0021500055.csv
0021500056.csv
0021500057.csv
0021500058.csv
0021500059.csv
0021500060

0021500547.csv
0021500548.csv
0021500549.csv
0021500550.csv
0021500551.csv
0021500552.csv
0021500553.csv
0021500554.csv
0021500555.csv
0021500556.csv
0021500557.csv
0021500558.csv
0021500559.csv
0021500560.csv
0021500561.csv
0021500562.csv
0021500563.csv
0021500564.csv
0021500565.csv
0021500566.csv
0021500567.csv
0021500568.csv
0021500569.csv
0021500570.csv
0021500571.csv
0021500572.csv
0021500573.csv
0021500574.csv
0021500575.csv
0021500576.csv
0021500577.csv
0021500578.csv
0021500579.csv
0021500580.csv
0021500581.csv
0021500582.csv
0021500583.csv
0021500584.csv
0021500585.csv
0021500586.csv
0021500591.csv
0021500592.csv
0021500593.csv
0021500594.csv
0021500595.csv
0021500596.csv
0021500597.csv
0021500598.csv
0021500599.csv
0021500600.csv
0021500601.csv
0021500615.csv
0021500616.csv
0021500617.csv
0021500618.csv
0021500619.csv
0021500620.csv
0021500621.csv
0021500622.csv
0021500623.csv
0021500624.csv
0021500625.csv
0021500626.csv
0021500627.csv
0021500628.csv
0021500629.csv
0021500630

In [18]:
def moments2JSON(moment):
    json = "{"
    json += " \"period\" :"+str(moment[0])+","
    json += " \"Unixtime\" :"+str(moment[1])+","
    json += " \"secondsRemaining\" :"+str(moment[2])+","
    json += " \"shotClock\" :"+str(moment[3])+","
    json += " \"misc\" : \""+str(moment[4])+"\","
    json += " \"positions\" :"+"["
    
    a=[]
    for positions in moment[5]:
        a.append("{ \"teamid\":"+str(positions[0])+", \"playerid\":"+str(positions[1])+", \"x\":"+str(positions[2])+", \"y\":"+str(positions[3])+", \"z\":"+str(positions[4])+"}")
        myString = ",".join(a)
    json += myString +"]}"
    return json

#convertedJSON = moments2JSON(momentsReversed[1])
#print convertedJSON
#a= "{
#    "period" : momentsReversed[0],
#    "Unixtime": momentsReversed[1],
#    "secondsRemaining": momentsReversed[2],
#    "shotClock": momentsReversed[3],
 #   "misc": momentsReversed[4],
#    "Unixtime": momentsReversed[5]  
#}"

#print myString

## Getting turn over data from missed alley oops

In [28]:
print("Starting unparsed pass dataset: "+ str(len(AlleyOop_df)))
CleanData_df = pd.DataFrame(columns=['GameID', 'Period', 'SportVUeventNum', 'SportVUMomentIndex', 'reversedIndexBallCaught', 'PBPGameTime' , 'timeballpassed','timeballcaught', 'ShotClock', 'PasserID', 'ReceiverID', 'PassDist',  'passInHull', 'receiverinHull', 'HullArea','receiverIsOpen' , 'PBP_HOMEDESCRIPTION' , 'PBP_VISITORDESCRIPTION', 'rawMoments', 'defensivePositions', 'isAlleyOOp', 'Success'])
momentsJson_df = pd.DataFrame(columns=['jsonMoments'])
if(not curPath.endswith('SportVU_JSON_Data')):
    curPath = os.getcwd()
    print curPath
    strIndex=curPath.index('\\JupyterNotebooks\\')
    curPath = curPath[:(strIndex+18)]+'SportVU_JSON_Data\\'
    os.chdir(curPath)
print curPath
#SportVU_JSON_Rootpath = "C:\Users\jchin\Documents\GitHub\Data Mining Course\DataMining-Project-Folder\JupyterNotebooks\SportVU_JSON_Data\\"
#print SportVU_JSON_Rootpath+str(gameid)+".json"
gameid_prev = 0;
for index, to in AlleyOop_df.iterrows():
    gameid = to["GAME_ID"]    #game id from PBP
    eventNumPBP = to["EVENTNUM"] #eventNumber for the play
    gametimePBP = to["PCTIMESTRING"]  #game time for the play
    quarterPBP = int(to["PERIOD"])
    #getPBPInfo(gameid,eventNumPBP)
    print("prevGameID:"+str(gameid_prev)+", currentlyAnalyzingGameID:"+str(gameid))
    print("PBPEventNum:"+str(eventNumPBP)+", gametimePBP:"+str(gametimePBP))
    #check if SportVU json data is already opened for this PBP play
    if(gameid_prev != gameid):
        #open raw JSON SportVU data using game id from assist
        with open(curPath+"00"+str(gameid)+".json") as data_file:    
            rawSportVUData = json.load(data_file)
            json_normalize(rawSportVUData['events'])
        gameid_prev = gameid
    
    #get the correct eventNum (returned as long) in SportsVU
    rawpossessionSportsVU, eventNumSportsVU = findCorrectMomentData(rawSportVUData, quarterPBP, gametimePBP, eventNumPBP)
    if(eventNumSportsVU is None):
        print("Could not find eventNum in SportsVU \n")
        continue
    
    print("Correct SportVU eventNum: "+str(eventNumSportsVU))
    #extract SportsVU movements for that eventNum
    
    rawpossessionSportsVU , SportsVUMomentIndex= MomentsfromEventNum(rawSportVUData, int(eventNumSportsVU))
    
    #for analyzing OOP only...
    receiverID = int(to['PLAYER2_ID'])  
    passerID = int(to['PLAYER1_ID'])
    teamID = int(to['PLAYER1_TEAM_ID'])
    if(math.isnan(to['PLAYER2_TEAM_ID'])):
        if (rawSportVUData['events'][0]['home']['teamid'] == teamID ):
            print("defense team: "+rawSportVUData['events'][0]['visitor']['name']);
            defenseTeamId =  rawSportVUData['events'][0]['visitor']['teamid'];
        else:
            defenseTeamId =  rawSportVUData['events'][0]['home']['teamid'];
            print("defense team: "+rawSportVUData['events'][0]['home']['name']);  
    else:
        defenseTeamId = int(to['PLAYER2_TEAM_ID'])
                 
    
    #PBPTime = str(oop['PCTIMESTRING'])
    PBP_HOMEDESCRIPTION = str(to['HOMEDESCRIPTION']) 
    PBP_VISITORDESCRIPTION = str(to['VISITORDESCRIPTION'])
    
                     
    #get catch details (who was the passer, receiver, and when caught/passed)
    timeBallReleased, indexBallReleased, timeBallCaught, indexBallCaught, timeBallPassed, indexBallPassed, receiverPosAtPass, receiverPosAtCaught, passerPosAtPass , SportVUMomentIndexIncorrect= getTOEventDetails(receiverID, passerID, defenseTeamId, gametimePBP, rawpossessionSportsVU)
    
    if(timeBallReleased is None):
        print("Could not find time for when ball was passed")
        continue
    
    #hull details
    momentsReversed = rawpossessionSportsVU[::-1]
    
    hull, defensive_positions = getHull(momentsReversed[indexBallCaught], defenseTeamId)
    
    #Defensive Hull Area
    hullArea = hull.area
    
    #passer in hull?
    passerInHull = isInHull(hull,defensive_positions, passerPosAtPass[0] ,passerPosAtPass[1])
                     
    #receiver in hull?
    receiverInHull = isInHull(hull,defensive_positions, receiverPosAtPass[0] ,receiverPosAtPass[1])
      
    #is receiver open?  ( np_points position of defense)
    closestDist, receiverIsOpen = is_open([receiverPosAtCaught[0],receiverPosAtCaught[1]], defensive_positions)
    
    #pass Distance
    passDist =distance.euclidean(passerPosAtPass,receiverPosAtCaught)
    
    #shot clock time
    shotClockTime = momentsReversed[indexBallPassed][3]
    
    rawMoments =  momentsReversed[indexBallCaught:indexBallPassed]
    #unreverse moments
    rawMoments = rawMoments[::-1]
 
    isAlleyOop = True;
    isSuccess = False;
    
    #PBPTime , PBP_HOMEDESCRIPTION , PBP_VISITORDESCRIPTION = getPBPInfo("00"+str(gameid), eventNumPBP) 
    CleanData_df.loc[len(CleanData_df.index)] = [gameid, quarterPBP, eventNumSportsVU, SportVUMomentIndex , indexBallCaught, gametimePBP, timeBallPassed, timeBallCaught, shotClockTime, passerID, receiverID , passDist, passerInHull, receiverInHull, hullArea, receiverIsOpen  , PBP_HOMEDESCRIPTION , PBP_VISITORDESCRIPTION, rawMoments, defensive_positions, isAlleyOop,  isSuccess]
    #momentsJson_df.loc[len(momentsJson_df.index)] = [jsonMoments]
    
CleanData_df.to_csv("CleanedDataMissedAlleyOOP_.csv", sep=',')
#momentsJson_df.to_csv("momentdataonly.csv", sep=' ', index=False, header=False)


Starting unparsed pass dataset: 226
D:\DataMining-NBA-Repo\JupyterNotebooks\SportVU_JSON_Data
D:\DataMining-NBA-Repo\JupyterNotebooks\SportVU_JSON_Data\
prevGameID:0, currentlyAnalyzingGameID:21500011
PBPEventNum:130, gametimePBP:10:30
Finding correct SportVU eventNum for gameid:0021500011
curr eventNum:130 , prev eventNum:None
Currently checking eventNumPBP: 130
SportVU Index for the eventNumPBP: 119


TypeError: int() argument must be a string or a number, not 'list'